In [ ]:
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel, GenerationConfig
import warnings

# Initialize tools
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Do this: pip install google-cloud-bigquery-storage and db_dtypes
# Suppress warnings with a specific message
warnings.filterwarnings(
    "ignore", 
    message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project."
)

In [ ]:
def generate_response(prompt, model_name="gemini-2.0-flash", temperature=0.4, top_k=30, top_p=0.3):
    ai_client = GenerativeModel(
        model_name=model_name,
        generation_config=GenerationConfig(
            temperature=temperature,  
            top_k=top_k,  
            top_p=top_p
        )
    )
    response_result = ai_client.generate_content(prompt)
    return response_result.candidates[0].content.text

In [ ]:
# Setup

project_name = "snowplow-cto-office"
bigquery_client = bigquery.Client(project=project_name)
product = "smartbill"
app_id = "smartbill"
#product = "vnet_autopay"
#product = "eplus"
dataset_id = f"""snowplow-cto-office.snowplow_{product}"""
#full_responses_table_name = f"""snowplow-cto-office.snowplow_{product}.Vismanet_AutoPay_events_filtered_L60D"""

table_names = [f"{app_id}_simple_urlpath_sequences", f"{app_id}_events_custom_T60D", f"{app_id}_page_views_aggregated", f"{app_id}_session_chunks", f"{app_id}_simple_title_sequences"]

In [ ]:
full_responses_table_name = f"""{dataset_id}.{app_id}_session_chunks"""
#Vismanet_AutoPay_events_filtered_L60D
query_headers = f"""SELECT
        column_name AS field_name,
        data_type AS field_type,
        description AS field_description,
    FROM
        `{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE
        table_name = '{app_id}_session_chunks';
    """

query_job = bigquery_client.query(query_headers)
table = query_job.result()

rows = list(table)

all_fields_type = {row[0]: row[1] for row in rows}
all_fields_desc = {row[0]: row[2] for row in rows}
all_fields_names = {row[0]: row[0] for row in rows}

In [ ]:
summary_list = {}
for i in table_names:
    query_tables_summary = f"""SELECT
        *
    FROM
        `{dataset_id}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE
        option_name = 'description'
        AND table_name = '{i}';
    """
    query_job = bigquery_client.query(query_tables_summary)
    table = query_job.result()
    rows = list(table)
    summary_list[f"{i}"] = rows[0][5]

In [ ]:
print(summary_list)

In [ ]:
def field_selection(bigquery_client, prompt, keywords, assumptions = None):
    print("Bigquery start.")
    #query_job = bigquery_client.query(query_headers)
    #table = query_job.result()
    # headers_values = []
    #for row in table:
    #    headers_values.append(row[:])
    text_with_headers = f"""

    **Prompt:**
    - {prompt}

    **Assumptions about prompt:**
    - {assumptions}

    **Table schema:**
    - Name of fields: {all_fields_names}
    - Data type of values in the fields: {all_fields_type}
    - Field descriptions: {all_fields_desc}

    **Instructions:**
    1. Identify and list all relevant fields from the table schema that are useful for answering the prompt.
    2. Use these keywords to find the relevant fields: {keywords}.
    3. Do not include any extraneous symbols, such as JSON formatting, brackets, or custom fields.
    4. For each selected field, explain its relevance and how it helps to answer the prompt.
    5. Make use of assumptions if they exist. The assumptions are correct and is incorporated into the prompt.

    **Important:**
    - If **all necessary fields** to answer the prompt are present in the schema, proceed to list the fields and their corresponding use cases: field_name, [use case]
    - If the prompt mentions a clearly defined concept or event that is missing AND cannot be calculated or derived from the available data, return only: `False, [explanation]`
    - Do **not** make assumptions or workarounds unless explicitly allowed.
    - ABSOLUTELY DO NOT USE ANY CUSTOME FIELDS. CUSTOM FIELDS ARE NOT ALLOWED.

    Output format:
    field_name1, use case
    field_name2, use case
    ...

    Do **not** hallucinate or infer information.

    """

    print("Bigquery end.")
    fields_response = generate_response(
        text_with_headers, 
        temperature=0.05, 
        top_k=30, 
        top_p=0.3
    )

    print(fields_response)
    print("----------------------------------------------------------------------------------")


    lines = fields_response.split("\n")
    first_part = []
    second_part = []
    var_type = []


    for line in lines:
        if "," in line:
            first, second = line.split(",", 1) 
            first_part.append(first.strip())  
            second_part.append(second.strip())  

    for i in first_part:
        var_type.append(all_fields_type[i])

    fields_type = {
        first_part[i]: {
            'data_type': var_type[i],
        }
        for i in range(len(first_part))
    }

    fields_desc = {
        first_part[i]: {
            'description': second_part[i],
        }
        for i in range(len(first_part))
    }

    return fields_type, fields_desc

In [ ]:
def correct_sql_error(prompt, keywords, expected_result, sql_code, error, fields_type, fields_desc, strategy):
    final_prompt = f"""
    **Prompt:**
    - {prompt}

    **Important keywords:**
    - {keywords}

    **Strategy for creating SQL code:**
    - {strategy}

    **Expected result:**
    - {expected_result}
    
    ### **Original SQL Query (Faulty):**
    {sql_code}

    ### **Error Message:** 
    {error}

    ### **Table Details:** 
    - Table name: {full_responses_table_name}
    - Field data types: {fields_type}
    - Field descriptions: {fields_desc}

    ### **Task:** 
    - Review the query and update the query to solve the error while adhering to the prompt, keywords, strategy, and expected result:
    
    ### **Guidelines:** 
    - Look at the error and identify where in the code the error exists.
    - Look at the numbers [xx, xx]
    - Resolve the issue by identifying and changing the code appropriately.

    ### **Output:**  
    Please provide only the corrected SQL query. Ensure the corrected query is syntactically and logically sound, and adheres to the provided guidelines and constraints.
    """


    response = generate_response(
        final_prompt, 
        temperature=0.1, 
        top_k=40, 
        top_p=0.3
    )
    print(response)

    return response.replace("```sql", "").replace("```", "").strip()

In [ ]:
def verify(prompt, fields_type, fields_desc, query, expected_result, assumptions, strategy):
    final_prompt = f"""
    ### **Prompt:** 
    {prompt}

    ### **Assumptions:** 
    {assumptions}

    ### **Strategy for creating SQL code:** 
    {strategy}

    ### **SQL Query:** 
    {query}

    ### **Expected result:** 
    {expected_result}

    ### **Table schema:** 
    - Fields descriptions: {fields_desc}
    - Fields type: {fields_type}

    ### **Task:**  
    - Does this query align with the combination of the prompt, the expected result?
    - Will the syntax work for BigQuery?

    ### **Guidelines:**  
    - Assume the query is incorrect unless there is strong evidence it is correct.  
    - If answering "Yes," to a task, explicitly explain why.  
    - If answering "No," to a task, identify the issue and explain why it does not meet the requirement.  
    - Look at the query and see if the code makes sense and produces reasonable result.
    - Answer the task questions indepnedently from each other.
    - Follow the output format.
    
    ### **Output format:**  
    Yes/No. explanation
    Yes/No. explanation
    """
    

    response = generate_response(
        final_prompt, 
        temperature=0.1, 
        top_k=30, 
        top_p=0.3
    )

    print(response)

    lines = response.split("\n")
    first_part = []
    second_part = []
    for line in lines:
        if "." in line:  # Ensure there is a comma before splitting
            first, second = line.split(".", 1)  # Split only at the first comma
            first_part.append(first.strip())  # Store the first part (column name)
            second_part.append(second.strip())  # Store the second part (description)

    check_list = []
    for i in range(len(first_part)):
        if first_part[i] == "No":
            check_list.append(second_part[i])

    return check_list



In [ ]:
def result_interpretation(prompt, result, code):
    result_text = f"""
    ### **Prompt:**
    {prompt}  

    ### **SQL code:**   
    {code}  

    ### **Data:**   
    {result}  


    ### **Task:**  
    Analyze the provided dataset and generate a meaningful interpretation that answers the given prompt in a user-friendly manner.  

    ### **Steps:**  
    1. **Answer the question directly.** Provide a clear, data-driven response.  
    2. **Extract key insights.** Identify trends, anomalies, or correlations that are not immediately obvious.  
    3. **Use external knowledge when relevant.** If applicable, explain insights using common patterns or domain knowledge.  
    4. **Focus on the overall pattern** rather than unnecessary details unless a specific outlier is crucial.  
    5. **State relevant assumptions.** Only include assumptions that impact the interpretation of result (avoid discussing column names, data types, or table structures).  

    ### **Guidelines:**  
    - **Be concise.** Prioritize clarity over lengthy explanations.  
    - **Use a structured format.** Keep responses readable with bullet points and section headings.  
    - **Avoid redundant details.** Don’t restate information that is already evident from the dataset.  
    - **Ignore missing table schema details.** Focus on strategy, definitions, and calculations instead.  
    - **Assistant with a friendly tone.** Speak to the user as if this is the data you have to asnwer the prompt. Make objective statements and discuss uncertainties when possible.  
    - **BigQuery.** The data is extracted from BigQuery with (U.S weekday numbering convention e.g. Sunday is 1 and Monday is 2).
    - **Use the code for interpretation** The SQL code that extracts the data contains information which explains certain assumptions and why the data looks the way it does. Make use of the code when making assumptions.

    ### **Expected Output Format:**  
    1. **Best Answer to the Prompt** (direct response)  
    2. **Key Insights & Trends** (summary of findings)  
    3. **Relevant Assumptions** (only those impacting interpretation of the result)  
    """


    final_answer_response = generate_response(
        result_text, 
        temperature=0.3,  # More creativity
        top_k=40, 
        top_p=0.6
    )

    return final_answer_response

In [ ]:
def extract_keywords(prompt, assumptions = None):
    final_prompt = f"""

    **Prompt:** 
    {prompt}

    **Assumptions about prompt:** 
    {assumptions}

    **Guidelines:** 
    - Extract the most important keywords from the prompt. 
    - The keywords will be given to a LLM together with the prompt so that it can better understand the prompt and what need to be answered.
    - Choose relevant words in the prompt that need to be considered rather that conjuring up new ones. Do not include redundant keywords.
    - Make use of assumptions if they exist. The assumptions are correct and are meant to clarify any ambiguous word or concept. You can think of them as addition to the prompt.

    ** Output format:** 
    <[keyword1, keyword2, ...]>
    
    """

    keywords = generate_response(
            final_prompt, 
            temperature=0.2, 
            top_k=20, 
            top_p=0.5
        )

    print(keywords)
    return keywords

In [ ]:
def expected(prompt, fields_type, fields_desc, assumptions = None):
    final_prompt = f"""
    ### **Task:**   
    You are retrieving data from BigQuery to answer user queries. Given a prompt, state the expected result that the LLM should provide.  

    ### **Prompt:**   
    {prompt} 

    ### **Assumptions:**
    {assumptions}

    ### **Available table schema:**
    - Fields description: {fields_desc}
    - Fields type: {fields_type}

    ### **Instructions:**  
    - Use available data information from the fields to create the best possible expected result for the prompt.
    - Only state what the expected result should be with no explanations about it.
    - The expected result should reflect what the LLM should output after querying BigQuery. In essence, how the format and structure of the data should look like. 
    - Let the output be in concise sentences.
    - Make use of the assumptions when creating the expected result. However, only take the most reasonable parts that are relevant.

    **Output format:**  
    Expected result: <Your response here>
    """
    
    response = generate_response(
            final_prompt, 
            temperature=0.2,  # Ensures consistency
            top_k=20, 
            top_p=0.5
        )
    print(response)
    return response

In [ ]:
def create_sql_code(prompt, fields_type, fields_desc, strategy, keywords, expected_result, assumptions = None):
    text_with_headers = f""""
    **Prompt:** {prompt}

    **Table schema:** 
    - Data types: {fields_type} 
    - Descriptions: {fields_desc} 
    - Table: {full_responses_table_name}

    **Strategy:**
    - {strategy} 

    **Keywords:** 
    - {keywords}

    **Expected result:** 
    - {expected_result}

    **Assumptions about prompt:** 
    - {assumptions}

    **Constraints:** 
    - The query is sent to BigQuery. It needs to follow BigQuery syntax.
    - Fully follow the strategy
    - Fully create a query that aligns with the expected result
    - Keep query costs as low as possible

    ### **Task:**  
    - Generate the best possible BigQuery SQL query that:  
    1. Follows the strategy step-by-step without deviation. Treat the strategy as a strict blueprint; implement each step exactly as described.
    2. Exclude all Null Values from everywhere (`IS NOT NULL`)

    **Query Guidelines:**
    - Select only necessary fields; avoid extra data.
    - Use accurate column names; no transformations.
    - Prefer partition filtering over `EXTRACT()`. Use `DATE_TRUNC(date_column, DATE_PART)` correctly (e.g., `DATE_TRUNC(session_started_date, WEEK)`), without string literals.
    - Use approximate counts where possible.
    - Consider clustering for better filtering.
    - **Exclude NULL values** (`IS NOT NULL`).
    - Make use of the assumptions. The assumptions are correct and is incorporated into the prompt.
    - Order and keep the data user-friendly and answers the prompt.
    - Be careful of LIMIT expects an integer literal or parameter
    - The query should only give the user ONE table of result.

    Use standard SQL. The query will be sent to **BigQuery**, so DO NOT INCLUDE EXPLANATIONS. Make sure the query is VALID. Do not hallucinate.
    """

    sql_query_response = generate_response(
        text_with_headers, 
        temperature=0.1, 
        top_k=20, 
        top_p=0.2
    )

    return sql_query_response.replace("```sql", "").replace("```", "").strip()

In [ ]:
aiAgent = DataQueryAgent(project_name, bigquery_client, generate_response)

In [ ]:

prompt = "Where are most of our users from? we don't have information on country"
#prompt = "Which companies have the highest engagement?"
#prompt = "Which customers have the most sessions?"
prompt = "Based on the bigger customers; what days and times is the best for performing updates, maintenance?"
#prompt = "At what times and days are people starting sessions the least?"
#prompt = "Based on the bigger customers; what days and times do least sessions start?"
#prompt = "Based on the top 10 customers with the most users; what days and times are the best for performing updates, maintenance?"
#prompt = "How do system outages impact user behavior? "
#prompt = "How did the user behavior change between 30th december 2024, 31th december 2024, 1th january 2025 and 2nd january 2025? "
prompt = "How did the user behavior change between 2024 and 2025 for the months?"
#prompt = "Do performance issues correlate with a drop in activity?"
#prompt = "What combinations of browser and os are the worst according to performance?"
#prompt = "Based on user behavior, what will the peak activity hours be tomorrow?"
#prompt = "How many users used each product throughout january 2025?"
#prompt = "How does app performance affect user engagement and retention?"
#prompt = "Which devices or OS versions are experiencing the worst performance (slower load times, etc)? and total page load time"
#prompt = "Which devices and OS versions are experiencing the worst performance?"
#prompt = "Vilka OS system och enheter är sämst enligt performance?"
#prompt = "Can you see a pattern if you compare Network Latency, Page Load Fails, Page Loading Time and Device and browsers?"
#prompt = "for all windows user, which brare best based on performance? Give top 20"
#prompt = "for all windows user, is edge or chrome better for performance? Compare all versions of edge and chrome. also show number of occurences of the combinations"
#prompt = "for all windows user, is edge or chrome better for performance? filter out and look for browsers that contain the name either edge or chrome. Compare all versions." 
#prompt = "Can you calculate the DAU/WAU and WAU/MAU stickiness for the products for all months in 2024?"
#prompt = "Can you calculate the monthly stickiness for the products in 2024?"
#prompt = "Can you calculate the WAU/MAU stickiness for the products for all weeks in 2024?"
#prompt = "Calculate the weekly stickiness (defined as the percentage of weekly active users who used the product in the previous week) for each product in our catalog for every week of the year 2024. Provide the output as a table with the following columns: Week Number (2024), Product name, and Weekly Stickiness (%)?"
#prompt = "Are there any insights in the correlation between the size of cutsomers and have huge page load times?"
#prompt = "What are the network latency & slow load times observed during hours when users are most active"
#prompt = "Most common drop-off points"
prompt = "How much did the total session time increase or decrease in percentage and number between each month in 2025 compared to the months in 2024? Can you divide into user roles"
#prompt = "For all Windows users, is Edge or Chrome better according to performance? Look for browsers that contain the name either edge or chrome. Compare all versions."
#prompt = "What devices are mostly used? and what dimensions are they?"
#prompt = "For all Windows users, is Edge or Chrome better according to performance? Look for browsers that contain the name either edge or chrome. Compare all versions."
#prompt = "For all Windows users, is Edge or Chrome better according to performance? Look for browsers that contain the name either edge or chrome. Compare all versions of edge and chrome."
#USers from vestlandfylke has ued teh solution
#prompt = "Can you calculate the WAU/MAU stickiness for the products for all weeks in 2024?"
#prompt = "for all windows user, is edge or chrome better for performance? Compare all versions of edge and chrome."

response = aiAgent.process_prompt(prompt)
print(response)
#new_user_input = input("Ask your follow-up question")
#response = aiAgent.process_prompt(new_user_input)

In [ ]:
class DataQueryAgent:
    """
    A class responsible for processing user prompts to generate and validate SQL queries 
    using BigQuery, incorporating user clarifications and query refinement.
    """
    
    def __init__(self, project_id: str, bigquery_client, generate_response_fn):
        self.client = bigquery_client
        self.project_id = project_id
        self.generate_response = generate_response_fn
    
    def process_prompt(self, user_input):
        """Processes user input to generate, validate, and execute an SQL query."""
        
        # Extract initial query parameters
        keywords = extract_keywords(user_input)
        fields_type, fields_desc = field_selection(self.client, user_input, keywords)

        # Handle assumptions
        assumptions = self.make_assumptions(user_input, fields_type, fields_desc, keywords)
        assumptions = self.refine_assumptions(user_input, assumptions)
        print(assumptions)
        
        # Re-evaluate query parameters
        #user_input = self.update_prompt(user_input, keywords, assumptions)
        #print("Updated prompt: " + user_input)
        keywords = extract_keywords(user_input, assumptions)
        fields_type, fields_desc = field_selection(self.client, user_input, keywords, assumptions)

        expected_result = expected(user_input, fields_type, fields_desc, assumptions)

        #user_input = self.update_prompt(user_input, keywords, expected_result, assumptions)
        #print("Updated prompt: " + user_input)
        
        # Generate and validate SQL query
        sql_code, strategy = self.generate_valid_sql(user_input, fields_type, fields_desc, keywords, expected_result, assumptions)
        if not sql_code:
            return "Failed to generate a valid SQL query after 3 attempts."
        
        # Validate SQL query using BigQuery dry run
        valid_result, result_df = self.validate_sql_query(prompt, keywords, expected_result, sql_code, fields_type, fields_desc, strategy)
        if not valid_result:
            return "An error occurred. Please try again."
        
        # Process query results
        result_df = self.clean_results(result_df)
        interpretation = result_interpretation(user_input, result_df, sql_code)
        print(result_df.to_string())
        print(result_df)
        return interpretation
    
    def refine_assumptions(self, user_input, assumptions):
        """Asks the user to confirm or refine assumptions."""
        while True:
            user_clarifications = input(self.clarify_assumptions(assumptions))
            if user_clarifications.lower() in {"", "yes", "y"}:
                return assumptions
            assumptions = self.interpret_clarification(user_input, assumptions, user_clarifications)
    
    def generate_valid_sql(self, user_input, fields_type, fields_desc, keywords, expected_result, assumptions):
        """Attempts to generate a valid SQL query within a loop."""
        
        for attempt in range(3):
            strategy = self.aggregation_strategy(user_input, fields_type, fields_desc, keywords, expected_result, assumptions)
            print(strategy)
            sql_code = create_sql_code(user_input, fields_type, fields_desc, strategy, keywords, expected_result, assumptions)
            print("----------------------------------------------------------------------------------")
            print(attempt)
            print(sql_code)
            validation_errors = verify(user_input, fields_type, fields_desc, sql_code, expected_result, assumptions, strategy)
            if not validation_errors:
                return sql_code, strategy  # Return valid SQL
        
        return None, None

    def validate_sql_query(self, prompt, keywords, expected, sql_code, fields_type, fields_desc, strategy):
        """Performs a dry run of the SQL query to check validity."""
        job_config = bigquery.QueryJobConfig(dry_run=True)
        
        for attempt in range(3):
            print("----------------------------------------------------------------------------------")
            print(attempt + 10)
            try:
                query_job = self.client.query(sql_code, job_config=job_config)
                query_job.result()
                print(f"Query is valid. Estimated processing: {query_job.total_bytes_processed / 1e9:.2f} GB.")
                return True, self.client.query(sql_code).to_dataframe()
            except Exception as e:
                print(f"Query validation failed (Attempt {attempt + 1}/3): {e}")
                sql_code = correct_sql_error(prompt, keywords, expected, sql_code, e, fields_type, fields_desc, strategy)
                #sql_changes = correct_sql_error(prompt, expected, sql_code, e, fields_type, fields_desc)
                #sql_code = correct_sql_error2(prompt, keywords, expected, sql_code, e, fields_type, fields_desc, strategy, sql_changes)
        return False, None
    
    def clean_results(self, result_df):
        """Cleans up missing or empty values in the query results."""
        return result_df.map(lambda x: "Missing Data" if x in {"", "None"} else x).dropna()
    
    def clarify_assumptions(self, assumptions):
        return f"Are the following assumptions correct?\n\n{assumptions}"
    
    def interpret_clarification(self, prompt, assumptions, user_clarifications):
        clarification_prompt = f"""
        ### **Initial Prompt:** {prompt}
        ### **Current Assumptions:** {assumptions}
        ### **User Clarifications:** {user_clarifications}
        ### **Task:**
        - Update assumptions based on clarifications.
        - If no change is needed, return the current assumptions.
        """
        return self.generate_response(clarification_prompt, temperature=0.2, top_k=40, top_p=0.5)
    
    def make_assumptions(self, prompt, fields_type, fields_desc, keywords):
        """Generates initial assumptions based on input."""
        strategy_prompt = f"""
        ### **Prompt:** {prompt}
        ### **Keywords:** {keywords}
        ### **Table Info:**
        - Fields Type: {fields_type}
        - Fields Description: {fields_desc}
        - Table name: {full_responses_table_name}
        
        ### **Constraints:**
        - Use only given fields.
        - No external data or assumptions beyond the provided fields.
        
        ### **Task:**
        - Propose short, reasonable, and user-friendly assumptions to resolve potential ambiguities in the prompt, leveraging the table info implicitly.
        - Propose reasonable definitions, calculations and other clarifications on the prompt.

        ### **Guidelines:**
        - Base assumptions on the most likely intent of the prompt, and keywords, informed by available data patterns from the table.
        - Keep assumptions simple and verifiable by the user, who lacks table schema knowledge.
        - Remember that the user cannot read all assumptions if they are too long. That is why it is important to keep each assumptions short as punctual.
        - Output a numeric list.
        - Do NOT generate SQL code.
        
        ### **Output Format:**
        1. ...
        2. ...
        """
        return self.generate_response(strategy_prompt, temperature=0.1, top_k=20, top_p=0.3)
    
    def aggregation_strategy(self, prompt, fields_type, fields_desc, keywords, expected_result, assumptions):
        strategy_prompt = f"""
        ### **Prompt:** {prompt}
        ### **Keywords:** {keywords}
        ### **Expected Result:** {expected_result}
        ### **Table Details:**
        - Fields Type: {fields_type}
        - Fields Description: {fields_desc}
        ### **Assumptions:**
        - {assumptions}
        ### **Constraints:** 
        - Use only the fields in the specified table—no external data or tables.  
        - Base the strategy solely on the given prompt, keywords, expected result, and table details.  
        ### **Task:** 
        - Suggest specific SQL operations in a step-by-step tailored to the prompt, assumptions and expected result.

        ### **Guidelines:**
        - Have many steps so that nothing is lost.
        - Do not generate executable SQL.
        - Keep the strategy concise and focused—no fluff or extraneous steps.
        - Make sure to give steps to make the query as user friendly as possible.

         ### **Output:**   
        - **Step-by-Step Strategy:** ("Title/Purpose" - do this)  
        """

        return self.generate_response(strategy_prompt, temperature=0.1, top_k=40, top_p=0.5)
    
    def update_prompt(self, prompt, keywords, assumptions):

        strategy_prompt = f"""
        ### **Prompt:** {prompt}
        ### **Keywords:** {keywords}

        ### **Assumptions:**
        - {assumptions}
        ### **Task:** 
        - Update the prompt by considering the keywords, and assumptions.
        - By update, you should update it so that it aligns with the keywords, and assumptions.
        - Make it as short and concise as possible.
        - Everything in the original prompt is still important and you need to conider what is said.
        """
        
        return self.generate_response(strategy_prompt, temperature=0.2, top_k=40, top_p=0.5)